In [7]:
def isprime(n):
    
    # make sure n is a positive integer
    n = abs(int(n))
    # 0 and 1 are not primes
    if n < 2:
        return False
    # 2 is the only even prime number
    if n == 2:
        return True
    # all other even numbers are not primes
    if not n & 1:
        return False
    # range starts with 3 and only needs to go up the square root of n
    # for all odd numbers
    for x in range(3, int(n**0.5)+1, 2):
        if n % x == 0:
            return False
    return True
 
# Create an RDD of numbers from 0 to 1,000,000
sc = SparkContext('local','pyspark')
nums = sc.parallelize(xrange(1000000))
 
# Compute the number of primes in the RDD
print nums.filter(isprime).count()

78498


In [6]:
# coding:utf-8
import pyspark
from pyspark import SparkContext, SparkConf

In [2]:
from __future__ import print_function

import re
import sys
from operator import add

from pyspark.sql import SparkSession

In [2]:
def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)


def parseNeighbors(urls):
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]

In [1]:
import pyspark
from pyspark import SparkContext
sc = SparkContext('local','spark')

In [11]:
#// 假定鄰居頁面的List存儲為Spark objectFile
links = sc.parallelize(List((1,Seq(2,3,4)), (2,Seq(1,3)), (3,Seq(1)),(4,Seq(3)))).partitionBy(HashPartitioner(100)).persist()

#//設置頁面的初始rank值為1.0
ranks = links.mapValues(lambda v: v=1.0)

#//迭代10次
for i in range (11): 
    contributions = links.join(ranks).flatMap(case (pageId, (links, rank)) =>links.map(dest => (dest, rank / links.size)))
        #//注意此時的links為模式匹配獲得的值，類型為Seq[String]，並非前面讀取出來的頁面List
        
    
    #//簡化了的rank計算公式
    ranks = contributions.reduceByKey((x, y) => x + y).mapValues(0.15 + 0.85 * v)

ranks.saveAsTextFile("ranks")

SyntaxError: invalid syntax (<ipython-input-11-a46db50c3d34>, line 9)

In [1]:
from pyspark import SparkContext, SparkConf


def f(x):
    # print x
    list1 = []
    s = len(x[1][0])
    for y in x[1][0]:
        list1.append(tuple((y, x[1][1]/s)))
    # print list
    return list1

In [2]:
if __name__== "__main__":
    conf = SparkConf()
    conf.setMaster("local")
    conf.setAppName("PageRank")

    # 定義sparkContext
    sc = SparkContext(conf=conf)

    # 原始數據
    list = [('A', ('D',)), ('B', ('A',)), ('C', ('A', 'B')), ('D', ('A', 'C'))]

    # 必須轉換成key-values,持久化操作提高效率，partitionBy將相同key的元素哈希到相同的機器上，
    # 省去了後續join操作shuffle開銷
    # tuple () 元組
    pages = sc.parallelize(list).map(lambda x: (x[0],  tuple(x[1]))).partitionBy(4).cache()

    # 初始pr值都設置為1
    links = sc.parallelize(['A', 'B', 'C', 'D']).map(lambda x: (x, 1.0))

    # 開始迭代
    for i in range(1, 100):
        # join會把links和page按k合併，如('A',('D',))和('A',1.0) join之後變成 ('A', ('D',1.0))
        # flatMap調用了f函數，並把結果平鋪
        rank = pages.join(links).flatMap(f)

        # reduce
        links = rank.reduceByKey(lambda x, y: x+y)

        # 修正
        links = links.mapValues(lambda x: 0.15+0.85*x)


    # links.saveAsTextFile("./pagerank")
    j = links.collect()

    for i in j:
        print i

KeyboardInterrupt: 

In [3]:
if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Usage: pagerank <file> <iterations>", file=sys.stderr)
        exit(-1)

    print("WARN: vvvThis is a naive implementation of PageRank and is given as an example!\n" +
          "Please refer to PageRank implementation provided by graphx",
          file=sys.stderr)

    # Initialize the spark context.
    spark = SparkSession\
        .builder\
        .appName("PythonPageRank")\
        .getOrCreate()

    # Loads in input file. It should be in format of:
    #     URL         neighbor URL
    #     URL         neighbor URL
    #     URL         neighbor URL
    #     ...
    lines = spark.read.text(sys.argv[1]).rdd.map(lambda r: r[0])

    # Loads all URLs from input file and initialize their neighbors.
    links = lines.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().cache()

    # Loads all URLs with other URL(s) link to from input file and initialize ranks of them to one.
    ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))

    # Calculates and updates URL ranks continuously using PageRank algorithm.
    for iteration in range(int(sys.argv[2])):
        # Calculates URL contributions to the rank of other URLs.
        contribs = links.join(ranks).flatMap(
            lambda url_urls_rank: computeContribs(url_urls_rank[1][0], url_urls_rank[1][1]))

        # Re-calculates URL ranks based on neighbor contributions.
        ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)

    # Collects all URL ranks and dump them to console.
    for (link, rank) in ranks.collect():
        print("%s has rank: %s." % (link, rank))

    spark.stop()

WARN: vvvThis is a naive implementation of PageRank and is given as an example!
Please refer to PageRank implementation provided by graphx


AnalysisException: u'Path does not exist: file:/home/w/-f;'

In [8]:
#// Load the edges as a graph
graph = GraphLoader.edgeListFile(sc, "/home/w/spark-2.2.0-bin-hadoop2.7/data/graphx/followers.txt")
#// Run PageRank
ranks = graph.pageRank(0.0001).vertices
#// Join the ranks with the usernames
users = sc.textFile("/home/w/spark-2.2.0-bin-hadoop2.7/data/graphx/users.txt")
ranksByUsername = users.join(ranks).map(case (id, (username, rank)) => (username, rank))
#// Print the result
println(ranksByUsername.collect().mkString("\n"))

SyntaxError: invalid syntax (<ipython-input-8-0d8553941186>, line 7)

In [3]:
def computeContribs(neighbors,rank):
    for neighbor in neighbors: yield( neighbor, rank/len(neighbors) )

links = sc.textFile("tst001.txt").map(lambda line: line.split()).map(lambda pages: (pages[0],pages[1]))\
.distinct().groupByKey().persist()

ranks=links.map(lambda (page,neighbors): (page,1.0) )

NameError: name 'sc' is not defined

In [ ]:
pyspark example

In [4]:
from __future__ import print_function

import re
import sys
from operator import add

from pyspark import SparkContext

# Initialize the spark context.
sc = SparkContext(appName="PythonPageRank")

In [11]:
#002

#// 假设相邻页面列表以Spark objectFile的形式存储
#val links = sc.objectFile[(String, Seq[String])]("links")
                  #.partitionBy(new HashPartitioner(100))
                  #.persist()
list = [('A', ('D',)), ('B', ('A',)), ('C', ('A', 'B')), ('D', ('A', 'C'))]
links = sc.parallelize(list).map(lambda x: (x[0],  tuple(x[1]))).partitionBy(4).cache()

In [13]:
links.collect()

[('A', ('D',)), ('D', ('A', 'C')), ('C', ('A', 'B')), ('B', ('A',))]

In [14]:
#// 将每个页面的排序值初始化为1.0;由于使用mapValues，生成的RDD // 的分区方式会和"links"的一样
ranks = links.mapValues(lambda v:(v,1.0))


In [17]:
# 開始迭代
for i in range(1, 100):
    # join會把links和page按k合併，如('A',('D',))和('A',1.0) join之後變成 ('A', ('D',1.0))
    # flatMap調用了f函數，並把結果平鋪
    rank = pages.join(links).flatMap(f)

    # reduce
    links = rank.reduceByKey(lambda x, y: x+y)

    # 修正
    links = links.mapValues(lambda x: 0.15+0.85*x)


# links.saveAsTextFile("./pagerank")
j = links.collect()

for i in j:
    print(i)

SyntaxError: invalid syntax (<ipython-input-17-ee309e72f2b6>, line 18)

In [3]:
def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)


def parseNeighbors(urls):
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]

In [3]:
if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Usage: pagerank <file> <iterations>", file=sys.stderr)
        exit(-1)

    print("""WARN: This is a naive implementation of PageRank and is
          given as an example! Please refer to PageRank implementation provided by graphx""",
          file=sys.stderr)
    
    # Loads in input file. It should be in format of:
    #     URL         neighbor URL
    #     URL         neighbor URL
    #     URL         neighbor URL
    #     ...
    lines = sc.textFile(sys.argv[1], 1)

    # Loads all URLs from input file and initialize their neighbors.
    #links = lines.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().cache()
    list = [('A', ('D',)), ('B', ('A',)), ('C', ('A', 'B')), ('D', ('A', 'C'))]
    links = sc.parallelize(['A', 'B', 'C', 'D']).map(lambda x: (x, 1.0))

    # Loads all URLs with other URL(s) link to from input file and initialize ranks of them to one.
    #ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))
    ranks = links.mapValues( v => 1.0)

    # Calculates and updates URL ranks continuously using PageRank algorithm.
    for iteration in range(10):
        # Calculates URL contributions to the rank of other URLs.
        contribs = links.join(ranks).flatMap(
            lambda url_urls_rank: computeContribs(url_urls_rank[1][0], url_urls_rank[1][1]))

        # Re-calculates URL ranks based on neighbor contributions.
        ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)

    # Collects all URL ranks and dump them to console.
    for (link, rank) in ranks.collect():
        print("%s has rank: %s." % (links, rank))
    #links.take(3)


#sc.stop()

WARN: This is a naive implementation of PageRank and is
          given as an example! Please refer to PageRank implementation provided by graphx


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 4, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/usr/local/lib/python2.7/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/lib/python2.7/dist-packages/pyspark/rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/lib/python2.7/dist-packages/pyspark/rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/lib/pyt

In [8]:
links.take(3)

[('A', 1.0), ('B', 1.0), ('C', 1.0)]

In [4]:
j = links.collect()

for i in j:
    print(i)

('A', 1.0)
('B', 1.0)
('C', 1.0)
('D', 1.0)


In [24]:
for x in ranks:
    print(x)

TypeError: 'PipelinedRDD' object is not iterable

In [13]:
ranks.first()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 87.0 failed 1 times, most recent failure: Lost task 0.0 in stage 87.0 (TID 21, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/usr/local/lib/python2.7/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/lib/python2.7/dist-packages/pyspark/rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/lib/python2.7/dist-packages/pyspark/rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/lib/python2.7

In [4]:
sys.argv[1]

'-f'